In [33]:
from opty import Problem, create_objective_function, parse_free
import sympy as sp
import numpy as np
import scipy as sc
import time as tm
import pickle
import sympy.physics.mechanics as me
import sys
sys.path.insert(0, "..")
from importlib import reload
import equations as eq
reload (eq);

initPos = 'InitPosOptQuat'

motion_folder_list = ['Elevation','Abduction_rigged2','Abduction_rigged','Abduction','Steering']
motion_list = ['elevation.mat','abd_rigged.mat','abd_rigged.mat','abduction.mat','steering.mat']
weights_list = [100,150,200,250,300]

for i in range(len(motion_folder_list)):
    for iweight in range(len(weights_list)):
        motion_folder = motion_folder_list[i]
        motion_name = motion_list[i]
        struct_name = 'results_euler_QuatInit_'+str(weights_list[iweight])
        act_w = 1
        traj_w = weights_list[iweight]
        vel_w = 0.1

        model_struct = sc.io.loadmat('../Motions/'+motion_folder+'/OS_model.mat')
        data_struct = sc.io.loadmat('../data_model.mat')

        start = tm.time()
        MM,FO,q,u,fr,frstar,kindeq,xdot = eq.create_eoms_eul(model_struct,data_struct,initPos,derive = 'numeric',gen_matlab_functions = 0)
        TE,activations,TE_conoid = eq.polynomials_euler(model_struct,q,derive = 'numeric',model_params_struct = data_struct,initCond_name = initPos)

        time_to_create = tm.time() - start
        print(time_to_create)

        dict_vals,symlist, value_list = eq.create_parameters_dict(data_struct, initPos)
        x0 = data_struct['params'][initPos][0,0]['initCondEul'].item()
        x0t = list(x0.T[0])
        eoms_implicit = sp.Matrix(kindeq).col_join(fr+frstar+TE)
        import trajectory_lib as tr
        reload (tr);
        num_nodes = 101
        file = '../Motions/' + motion_folder + '/' + motion_name
        traj_original, interval_value, time = tr.exp_trajectory_eul(file,num_nodes)
        traj = tr.exp_trajectory_eul_myobj(traj_original)

        state_symbols = tuple(q+u)
        num_states = len(state_symbols)
        specified_symbols = tuple(activations)
        num_inputs = len(specified_symbols)
        t = me.dynamicsymbols._t
        objective_traj,objective_traj_jac = eq.custom_objective_eul(len(q),interval_value)

        def obj(free):
            # min_traj = traj_w * interval_value * np.sum((traj - free[:10*num_nodes])**2)
            min_traj = traj_w * np.sum(objective_traj(np.split(free[:10*num_nodes],10),traj))
            min_vel = vel_w * interval_value * np.sum((free[10*num_nodes:num_states*num_nodes])**2)
            min_torque = act_w * interval_value * np.sum(free[num_states*num_nodes:(num_states + num_inputs)*num_nodes]**2)
            return min_traj + min_torque + min_vel

        def obj_grad(free):
            grad = np.zeros_like(free)
            # grad[:10*num_nodes] = traj_w * 2.0 * interval_value * (free[:10*num_nodes] - traj)
            grad[:10*num_nodes] = traj_w * np.concatenate(objective_traj_jac(np.split(free[:10*num_nodes],10),traj))
            grad[10*num_nodes:num_states*num_nodes] = vel_w * 2 * interval_value * free[10*num_nodes:num_states*num_nodes]
            grad[num_states*num_nodes:(num_states + num_inputs)*num_nodes] = act_w * 2.0 * interval_value * free[num_states*num_nodes:(num_states + num_inputs)*num_nodes]
            return grad
        instance_constraints = []
        # for i in range(9):
        #     instance_constraints.append(state_symbols[i].func(0.0)-x0t[i])

        instance_constraints.append(state_symbols[-1].func(0.0)-0)

        bounds1 = (0.0,1.0)
        bounds = (bounds1,)*len(activations)
        bndrs = dict(zip(activations,bounds))
        start = tm.time()

        prob = Problem(obj, obj_grad, eoms_implicit, state_symbols,
                    num_nodes, interval_value,
                    known_parameter_map={},
                    instance_constraints=instance_constraints,
                    bounds=bndrs,
                    integration_method='midpoint'
        ) #               
        time_to_create = tm.time() - start
        print(time_to_create)
        prob.add_option('max_iter',2500)
        prob.add_option('limited_memory_max_history', 40)
        initial_guess = np.zeros(prob.num_free)
        initial_guess[:10*num_nodes] = traj_original.flatten()
        time_2_solve_start = tm.time()
        solution, info = prob.solve(initial_guess)
        time_2_solve = tm.time() - time_2_solve_start
        print(info['status_msg'])
        print(info['obj_val'])
        act_obj = np.sum(solution[num_states*num_nodes:(num_states + num_inputs)*num_nodes]**2)


        print('Objective activations: ', act_obj)
        import matplotlib.pyplot as plt
        fig, axes = plt.subplots(int(num_states+num_inputs), 1, sharex=True,
                                figsize=(6.4, 0.8*(num_states+num_inputs)),
                                layout='compressed')
        prob.plot_trajectories(solution, axes=axes)
        import matplotlib.pyplot as plt
        fig, axs = plt.subplots(10)
        for j in range(10):
            axs[j].plot(time,traj_original.flatten()[j*num_nodes:(j+1)*num_nodes])
            axs[j].plot(time,solution[j*num_nodes:(j+1)*num_nodes])
            fig.set_figheight(10)
        fig, axes = plt.subplots(2, figsize=(12.8, 9.6),
                                layout='constrained')
        prob.plot_constraint_violations(solution, axes=axes)
        import trajectory_lib as tr
        reload (tr);
        # num_iter_sol = int(input('enterr number of iterarions:'))
        file_name = '../Motions/'+motion_folder+'/' + struct_name + '.mat'
        tr.sol2struct(solution,activations,len(q),num_states,num_nodes,time,0,time_2_solve,file_name)

        file_name_mot = '../Motions/'+motion_folder+'/' + struct_name + '.mot'
        tr.sol2mot_eul(solution, num_nodes, len(q), time, file_name_mot)

KeyboardInterrupt: 